<h1> 2d. Distributed training and monitoring </h1>

In this notebook, we refactor to call ```train_and_evaluate``` instead of hand-coding our ML pipeline. This allows us to carry out evaluation as part of our training loop instead of as a separate step. It also adds in failure-handling that is necessary for distributed training capabilities.


In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [1]:
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

2.2.0-dlenv


<h2> Input </h2>

Read data created in Lab1a, but this time make it more general, so that we are reading in batches.  Instead of using Pandas, we will use add a filename queue to the TensorFlow graph.

In [16]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512):
    def _input_fn():  
        def decode_csv(row):
            columns = tf.io.decode_csv(row, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            features.pop('key') # not needed, since it's not a feature
            label = features.pop('fare_amount') # remove label from features and store
            return features, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        filenames_dataset = tf.data.Dataset.list_files(filename, shuffle=False)
        # Read lines from text files
        textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = textlines_dataset.map(decode_csv)

        # Note:
        # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
        # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # loop indefinetly
            dataset = dataset.shuffle(buffer_size = 10 * batch_size, seed = 42)
        else:
            num_epochs = 1 # one run and it's over

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset
    
    return _input_fn

<h2> Create features out of input data </h2>

For now, pass these through.  (same as previous lab)

In [13]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
    # Nothing to add (yet!)
    return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Serving input function </h2>

In [17]:
# Defines the expected shape of the JSON feed that the model
# will receive once deployed behind a REST API in production.
def serving_input_fn():
    serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        tf.feature_column.make_parse_example_spec(INPUT_COLUMNS))
    serving_input_receiver = serving_input_fn()
    return serving_input_receiver

<h2> tf.estimator.train_and_evaluate </h2>

In [18]:
def train_and_evaluate(output_dir, num_train_steps):
    #tf.compat.v1.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    tf.python.summary.FileWriterCache.clear()
    
    # DONE: Create tf.estimator.DNNRegressor train and evaluate function passing args['parsed_argument'] from task.py
    opti = tf.keras.optimizers.Adam(learning_rate = 0.01)
    estimator = tf.estimator.DNNRegressor(
        model_dir = output_dir,
        feature_columns = feature_cols,
        hidden_units = [32, 8, 2],
        optimizer=opti)

    train_spec=tf.estimator.TrainSpec(
        input_fn = read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = num_train_steps)
    
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

    eval_spec=tf.estimator.EvalSpec(
        input_fn = read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL),
        steps=None,
        start_delay_secs = 1, # start evaluating after N seconds
        throttle_secs = 10,  # evaluate every N seconds
        exporters = exporter)
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    

<h2>Run training</h2>

In [19]:
OUTDIR = './taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.python.summary.FileWriterCache.clear()
train_and_evaluate(OUTDIR, num_train_steps = 500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distribu

## Challenge Exercise

Modify your solution to the challenge exercise in c_dataset.ipynb appropriately.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License